# 一、预处理
## 1. 分词以及停用词去除
    这个步骤的主要作用是：
```text
1. 将所有文本进行分词形成语料库
2. 剔除语料库中没有意义的停用词，如：我，我们，我的，他的等词

In [ ]:
import pandas as pd
from pathlib import Path
import jieba.posseg as psg
import re

### 1.1 加载原始文本数据

In [15]:
file_path = Path().resolve().parent/'resource'/'data'/'raw'/'weibo_keyword.csv'

# # 检测文件编码
# import chardet
# with open(file_path, 'rb') as f:
#     result = chardet.detect(f.read())
#     encoding = result['encoding']
# print(encoding)

# 只读取text列的前5000行作为实验性分析
df = pd.read_csv(file_path, encoding='GB2312',encoding_errors='ignore', usecols=['text'], nrows=5000).astype(str)

In [17]:
# 检查数据读取情况
print(df.info,df.columns)

<bound method DataFrame.info of                                                    text
0              是哪个大学生这么倒霉哦原来是我一边浑身不舒服发烧一边还要为期末考试焦头烂额新冠你
1     每日早报2022年07月26日星期二农历六月二十八1最高法加大对文娱领域高净值人群逃税惩处力...
2     我的爷爷从我记事起就一直在吃药从小跟着爷爷奶奶生活的我很幸福没有受过什么委屈上学会送我放假也...
3     252022下半年刷了一篇德国任职37岁大学教授文。大学是什么？是人类先锋某一个学生在某个时...
4                           大学生很好年轻朝气但心智不成熟适合校园恋爱。不适合我。
...                                                 ...
4995  防范电信诈骗网恋女友竟是抠脚大汉日前一男子因欠下巨额债务便冒充女大学生与一名男子谈恋爱诈骗钱...
4996  小时候的王俊凯太可爱了王俊凯在微博里说自己成绩不好因为作业极度伤心！王俊凯从小到大都是品学兼...
4997  未来你好我刚刚在大学生最喜爱的音乐人评选活动为小鬼王琳凯投出一票！快来参加为你最爱的音乐人打...
4998  2020中国大学生好创意为中国加油！武汉加油！全国大广赛组委会作者刘毅李竹青学校百色学院让我...
4999  同心战疫为爱而歌2020年2月日本音乐家吉田携手湖北襄阳籍的男歌手李行亮及湖北武汉籍的演员音...

[5000 rows x 1 columns]> Index(['text'], dtype='object')


### 1.2 读取停用词

In [21]:
stop_word_path = Path().resolve().parent/'resource'/'stop_words'/'hit_stopwords.txt' # 哈工大停词表
supplement_path = Path().resolve().parent/'resource'/'stop_words'/'supplement.txt' # 根据分词结果，补充停词表
try:
    with open(stop_word_path, encoding='utf8') as file:
        stop_list = {word.strip() for word in file if word.strip()}
        print("成功读取哈工大停词表，总数为", len(stop_list))
    with open(supplement_path, encoding='utf8') as file:
        stop_list.update({word.strip() for word in file if word.strip()})
        print('成功更新停词表，此时总数为', len(stop_list))
except FileNotFoundError:
    stop_list = set()
    print(f'Error: {stop_word_path} 没找到')

成功读取哈工大停词表，总数为 749
成功更新停词表，此时总数为 753


### 1.3 调用jieba.posseg.cut()函数进行分词并根据词性只保留名词类

    在psg.cut(text)函数中
```text 
1. 只接受字符串类型的参数，不能为DataFrame或者Series，如果数据存储为这两种，需要遍历后再作为参数传递。
2. 返回值为生成器(generator)，对于生成器可以用循环直接迭代
3. 生成器是迭代器的特殊形式，按需生成数据，而非一次生成所有数据，有利于降低内存占用，适合处理流式数据以及需要按行处理的数据。

#### 方法：定义分词函数

In [35]:
def word_cut(sr): # 此处text为字符串，而非DataFrame或Series
    word_list = []
    flag_set = ['n', 'vn', 'nz']
    for string in sr:
        row_word = []
        for seg_word in psg.cut(string): # 循环迭代 psg.cut(text)产生的生成器
            word = re.sub(r'[^\u4e00-\u9fa5]', '', seg_word.word)
            if word and len(word) > 1 and seg_word.flag in flag_set and word not in stop_list:
                row_word.append(word)
        word_list.append((' ').join(row_word))
    return word_list
df['word_cutted'] = word_cut(df.text)

# 保存分词结果，下次使用时直接读取
file_save_path = Path().resolve().parent/'resource'/'data'/'processed'/f'{Path(file_path).stem}_cutted{Path(file_path).suffix}'
df.to_csv(file_save_path,index=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\persi\AppData\Local\Temp\jieba.cache
Loading model cost 0.797 seconds.
Prefix dict has been built successfully.


# 二、训练LDA模型
    使用 gensim 库中的 LdaModel 来训练一个主题模型
## 1. 创建字典
```text   
因为 LDA 模型在内部处理时需要使用数字而不是文本数据，因此通过 corpora.Dictionary() 创建一个字典，这个字典会为每个唯一的词分配一个整数 ID。
corpora.Dictionary()函数需要一个文档集合作为输出，其中每个文档都是词汇列表的形式。也即，该函数需要传递一个嵌套的列表作为输入，其中最内层的列表包含一段文本的分词结果。而在上面定义的word_cut()函数返回的值为字符串，因此需要现将字符串分割为列表，用到的方法为split()

In [ ]:
from gensim import corpora
df['word_cutted'] = df['word_cutted'].apply(lambda x: x.split())
dictionary = corpora.Dictionary(df['word_cutted'])
dictionary.token2id

## 2. 建立词袋
```text 
使用 dictionary.doc2bow() 方法为每个文档创建一个词袋（bag-of-words, BOW）。这个方法统计每个唯一词的出现次数，并将文本转换为一个稀疏向量。稀疏向量是指数据中为零的数据远多于非零数据。

In [ ]:
corpus = [dictionary.doc2bow(text) for text in df.word_cutted]
corpus

## 3. 训练LDA模型
`通过ldamodel.LdaModel()函数来训练LDA模型，其中包含7个主要的参数：`
```text 
1. corpus：文档的词袋表示
2. id2word：词典
3. num_topics：主题的数量

4. alpha：
alpha 参数是与文档-主题分布相关的狄利克雷先验的参数。它决定了文档中主题的分布：
高 alpha 值：意味着每个文档很可能包含多个主题，即每个文档中的主题分布比较均匀。这会导致文档包含更多的主题，每个文档的主题分布较为平滑。
低 alpha 值：意味着每个文档倾向于由少数几个主题主导。这导致每个文档的主题分布较为集中，即文档中的主题数量较少。
alpha 的选择取决于对文档主题多样性的假设。在不确定时，可以通过模型选择方法（如交叉验证）来确定最优的 alpha 值。（可以设置为 'auto' 以自动学习）

5. eta：
eta 参数是与主题-词分布相关的狄利克雷先验的参数。它决定了每个主题中词的分布：
高 eta 值：每个主题包含的词汇更均匀分布。这意味着每个主题将具有较广泛的词汇。
低 eta 值：每个主题被少数几个词主导。这会导致每个主题的词分布较为集中，即主题专注于较少的词汇。
与 alpha 类似，eta 的设定应基于对主题词分布的假设，而实际值通常通过实验来确定。（可以设置为 'auto' 以自动学习）

6. passes：代表语料库要被重复遍历的次数以进行模型训练。每遍历一次完整的语料库称为一遍“pass”。
7. random_state：控制随机性的种子，可以使得在不同运行中复现结果。

In [ ]:
from gensim.models import ldamodel
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, alpha=0.1, eta=0.01, passes=100, random_state=2)
lda.save(r'resource/result/lda.model')

## 4. 调用LDA模型并打印每个主题最重要的词

In [ ]:
def print_lda_topics(lda_model, dictionary, n_top_words):
    tword = []
    for topic_idx in range(lda_model.num_topics): # 遍历所有主题
        topic_word = []
        topic_str = 'topic ' + str(topic_idx)
        topic_word.append(topic_str)
        print("Topic #%d:" % topic_idx)
        topic_terms = lda_model.get_topic_terms(topic_idx, topn=n_top_words)
        topic_w = " ".join([dictionary[term_id] for term_id, _ in topic_terms])
        topic_word.append(topic_w)
        tword.append(topic_word)
        print(topic_w)
    return tword

lda = ldamodel.LdaModel.load(r'./resource/result/lda.model')
n_top_words = 30  # 每个主题打印多少词语
topic_word = print_lda_topics(lda, dictionary, n_top_words)
topic_word_table = pd.DataFrame(data=topic_word, columns=['主题', '主题前30个高频词'])
topic_word_table.to_excel('topic_word_table.xlsx', index=False)

In [ ]:
topic_word_table

## 5. 调用大语言模型构建主题标识类别

In [ ]:
import configparser
qianfan_config_path = 'resource/qianfan-config.ini'
config = configparser.ConfigParser()
config.read(qianfan_config_path, encoding='utf8')
AK = config['qianfan']['ak']
SK = config['qianfan']['sk']

# 通过环境变量传递（作用于全局，优先级最低）
import os
os.environ["QIANFAN_ACCESS_KEY"] = AK
os.environ["QIANFAN_SECRET_KEY"] = SK

import qianfan
row_data = pd.read_excel('topic_word_table.xlsx')
data = row_data['主题前30个高频词']
chat = qianfan.ChatCompletion()
response = chat.do(model='ERNIE-Speed-128k', messages=[
    {
        "role": "user",
        "content": f"请根据以下数据,为每一行提炼一个标题。要求如下：1. 每个标题的字数和格式必须一致。2. 标题不能包含“……主题”、“……专题”或“……高频词汇”。3. 每行的总结必须独立输出，不得合并或遗漏。4. 以JSON格式输出。5. 仅输出主题标识类别及其对应内容，无需额外解释。数据如下：{data}"
    }

])

print(response['body']['result'])

`对于自定义的print_lda_topics函数：`
`参数:`
```text 
lda_model: 这是经过训练的 LDA 模型对象。
dictionary: 这是一个 gensim 字典对象，用于将词的 ID 转换为词本身。
n_top_words: 指定要显示的每个主题的关键词数量。
```

`流程:`

```text 
1. 函数遍历模型中的每个主题。
2. 对于每个主题，使用 get_topic_terms 方法提取前 n_top_words 个最重要的词。
3. 将这些词的 ID 转换为词本身，并拼接成一个字符串。
4. 打印并收集每个主题的关键词字符串。
```

`返回值:`

```text 
返回一个列表，其中每个元素是一个代表主题的字符串，包含了该主题的前 n_top_words 个关键词。
```
`代码注解：`

`1. tword = [ ]`

```text 
初始化一个空列表，用来存储每个主题的关键词列表。最终，这个列表将包含每个主题的关键词字符串，每个字符串由该主题的前 n_top_words 个最重要的词组成。
```
`2. for topic_idx in range(lda_model.num_topics):`

```text 
lda_model.num_topics 方法返回lda模型的主题数，得到一个数字。所以range(lda_model.num_topics) 本质上就是range(number) 。

range(number)是range(start, end, sep)的缩写形式，此时end=number。因此，range(lda_model.num_topics) 生成一个从 0 到 lda_model.num_topics - 1 的整数序列，代表所有的主题编号。形式为[0,1,2,3……]

整个for循环语句就用于循环遍历所有主题
```
`3. print("Topic #%d:" % topic_idx)`

```text 
打印：Topic #0：Topic #1：…… Topic #7:
```

`4. lda_model.get_topic_terms(topic_idx, topn=n_top_words)`

```text 
获取当前主题的前 n_top_words 个最重要的词。这个方法返回一个列表，其中的元素是 (word_id, probability) 对，表示词的ID和该词在主题中的权重。
```

`5. " ".join([dictionary[term_id] for term_id, _ in topic_terms])`
```text 
[dictionary[term_id] for term_id, _ in topic_terms]

这部分是一个列表推导式，用于从 topic_terms 列表中提取每个元素的 term_id，然后使用这个 term_id 从 dictionary 对象中查找相应的词。

topic_terms 是一个列表，其中的每个元素是一个由 (term_id, probability) 组成的元组。这个元组表示在当前主题中，某个词（由 term_id 标识）的重要性（由 probability 表示）。

term_id, _ 这部分代码使用 Python 的解构（unpacking）功能来提取元组中的 term_id。下划线 _ 在这里用作占位符，表示忽略第二个元素（probability）。

dictionary[term_id]

这是一个通过 term_id 从 dictionary 中获取对应词汇的操作。dictionary 是一个 gensim.corpora.Dictionary 对象，它存储了整个语料库中所有词的索引和映射。

" ".join(...):

join 方法将一个字符串列表合并成一个单一的字符串，其中原来的列表元素通过指定的分隔符（这里是空格 " "）连接起来。
这一步将上面列表推导产生的所有词合并为一个字符串，每个词之间用空格分隔。这样生成的字符串为当前主题的直观文本表达，列出了该主题的关键词。
```
`6. tword.append(topic_w)`
```text 
将构建好的字符串（包含主题中的关键词）添加到列表 tword 中
```

## 5. 可视化主题结果

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
def lda_visualization(lda, corpus, dictionary):
    vis_data = gensimvis.prepare(lda, corpus, dictionary)
    pyLDAvis.display(vis_data)
    pyLDAvis.save_html(vis_data, 'lda_pass.html')
lda_visualization(lda, corpus, dictionary)

# 三、模型优化
## 1. 结合TF-IDF优化词袋corpus
```text 
TF-IDF可以帮助我们识别出语料库中最重要的词汇。这些词汇通常对文档的主题具有很高的判别力。通过这个步骤，我们可以减少需要考虑的词汇数量，从而降低模型的复杂性和计算成本。
```

In [ ]:
# 用tfidf优化词袋
from gensim import models
tfidf = models.TfidfModel(corpus)
corpus = tfidf[corpus]

# 用优化后的词袋重新训练LDA模型
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,  alpha=0.1, eta=0.01,random_state=2,passes=100)

# 调用优化后的模型展示主题并可视化
n_top_words = 30  
topic_word = print_lda_topics(lda, dictionary, n_top_words)
lda_visualization(lda, corpus, dictionary)

## 2. 确定主题数量
```text 
主题数量太少不能捕捉数据中所有的细节信息，但是主题数量太多又会增加模型的解释难度，因此主题数量的确定需要在模型的准确性和解释的复杂性之间寻求平衡。

如何让主题数量在合理的范围从而控制解释的复杂性相对主观性强一些，可以灵活变通。但模型的准确性有相对客观的模型评价指标来进行量化。一般主要考虑两个指标：困惑度和主题一致性。
```
- `困惑度是衡量概率模型预测能力的一种指标，它的核心思想是，一个好的模型应该能很好地预测看不见的数据。`
- `较低的困惑度意味着模型预测新数据的能力更强，在统计上更可能生成观测到的数据。`

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

def perplexity(topics_num_range):
    perplexity_list =[]
    for num_topics in topics_num_range:
        lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=100,alpha=0.1, eta=0.01,random_state=2)  
        log_perplexity = lda_model.log_perplexity(corpus)
        perplexity = np.exp2(-log_perplexity)
        perplexity_list.append(perplexity)
    plt.plot(topics_num_range, perplexity_list)
    plt.xlabel('主题数目')
    plt.ylabel('困惑度大小')
    plt.rcParams['font.sans-serif']=['SimHei']
    matplotlib.rcParams['axes.unicode_minus']=False
    plt.title('主题-困惑度变化情况')
    plt.show()

perplexity(range(5,15))

## 3. 主题一致性
- `主题一致性度量一个主题的词在语义上是否相关。`
- `高一致性得分通常表明主题的词汇在语义上是相关的，这使得主题更容易被人理解和解释。`

In [ ]:
# 一致性
def coherence_lda(num_topics_range, corpus, texts, dictionary ):
    coherence_dict =dict()
    coherence_list =[]
    for num_topics in num_topics_range:
        lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=100,alpha=0.1, eta=0.01,random_state=1)  
        coherence = models.CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='u_mass').get_coherence()  # coherence 参数指定了计算一致性得分的具体方法。'u_mass' 是其中一种常用的一致性评价方法。
        coherence_dict['num_topics'] = coherence
        coherence_list.append(coherence)
    plt.plot(num_topics_range,coherence_list)
    plt.xlabel('主题数目')
    plt.ylabel('coherence大小')
    plt.rcParams['font.sans-serif']=['SimHei']
    matplotlib.rcParams['axes.unicode_minus']=False
    plt.title('主题-coherence变化情况')
    plt.show()
    return coherence_dict

coherence_dict = coherence_lda(num_topics_range=range(2,6), corpus=corpus, texts=df.word_cutted, dictionary=dictionary)
max_coherence_values = max(coherence_dict.
                           values())
max_coherence_key = [key for key, values in coherence_dict.items() if values == max_coherence_values]
print(
    f'主题{max_coherence_key}的coherence值最大')


In [ ]:
best_lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, alpha=0.1, eta=0.01, passes=100, random_state=2)
lda_visualization(best_lda, corpus, dictionary)

In [ ]:
import pandas as pd

def get_document_topics_and_max(lda_model, corpus):
    # 初始化列表，用于存储每个文档的主题概率和最可能的主题
    all_topics = []
    most_likely_topics = []
    most_likely_topics_prob = []
    
    # 遍历每个文档，获取主题分布
    for doc_bow in corpus:
        # 获取文档的所有主题及其概率，确保即使概率为0也能获取到所有主题
        doc_topics = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        # 使用字典存储当前文档的所有主题和相应的概率
        topic_probs = {f"Topic_{int(topic_id)}": prob for topic_id, prob in doc_topics}
        
        # 找出概率最大的主题和概率值
        max_topic, max_prob = max(doc_topics, key=lambda item: item[1], default=(None, 0))
        topic_probs['Most_Likely_Topic'] = int(max_topic) if max_topic is not None else None
        topic_probs['Most_Likely_Topic_Prob'] = max_prob
        
        # 将主题概率字典添加到列表中
        all_topics.append(topic_probs)
        most_likely_topics.append(int(max_topic) if max_topic is not None else None)
        most_likely_topics_prob.append(max_prob)
    
    return all_topics, most_likely_topics, most_likely_topics_prob

# 假设 lda 和 corpus 已经定义
all_doc_topics, most_likely_topics, most_likely_topics_prob = get_document_topics_and_max(lda, corpus)

# 创建新的DataFrame来存储所有主题和概率
topics_df = pd.DataFrame(all_doc_topics)

# 合并原始DataFrame和新创建的topics DataFrame
df = pd.concat([df, topics_df], axis=1)

# 保存更新后的DataFrame到Excel文件
df.to_excel("updated_document_topics.xlsx", index=False)
print("文档及其主题概率已成功更新并保存到 updated_document_topics.xlsx")


In [ ]:
def get_document_topics(lda_model, corpus):
    topics_distribution_list = []
    for doc in corpus:
        topics_distribution = lda_model.get_document_topics(doc, minimum_probability=0)
        topics_distribution_list.append(topics_distribution)
    return topics_distribution_list
topics_distribution_list = get_document_topics(lda, corpus)

In [ ]:
topics_info = []
for doc_id, doc_dist in enumerate(topics_distribution_list):
    doc_topics_info = [y for x,y in doc_dist]
    max_doc_topics = max(doc_dist, key= lambda x: x[1])[0]
    doc_topics_info.append(max_doc_topics)  
    topics_info.append(doc_topics_info)
topics_info

data = pd.DataFrame(data=topics_info, columns=['#0','#1','#2','#3','#4','所属主题'])
data.index.name = '文档编号'
data

In [ ]:
data.to_excel('topics_distribution.xlsx',index=True)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

font_path = "resource/SimHei.ttf"


def plot_topic_wordcloud(lda_model, dictionary, num_topics):
    for topic_id in range(num_topics):
        plt.figure(figsize=(12,8))
        topic_terms = lda_model.get_topic_terms(topic_id, topn=30)
        topic_words = {dictionary[word_id]: prob for word_id, prob in topic_terms}
        
        # 指定 font_path 以支持中文显示
        wordcloud = WordCloud(font_path=font_path, width=1200, height=600, background_color='white').generate_from_frequencies(topic_words)
        
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.title(f'Topic #{topic_id + 1}')
        plt.show()

# 调用函数
plot_topic_wordcloud(lda, dictionary, num_topics=7)


In [ ]:
topics_info

# 四、计算主题时间强度变化趋势

In [ ]:
# 计算日期-主题强度
def get_document_topics(lda_model, corpus):
    doc_topics = lda_model.get_document_topics(corpus, minimum_probability=0)
    return doc_topics

doc_topics = get_document_topics(lda, corpus)

# 添加日期列并转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 创建时间段列，按十年分段
def get_decade(date):
    year = date.year
    # 5年
    period_start = (year // 5) * 5
    return f"{period_start}-{period_start + 4}"

    # #10年 
    # decade_start = (year // 10) * 10
    # return f"{decade_start}-{decade_start + 9}"

df['decade'] = df['date'].apply(get_decade)

# 初始化主题强度数据结构
topic_strength = {i: [] for i in range(lda.num_topics)}

# 遍历文档，计算每篇文档的主题分布并按时间段聚合
for doc_id, (decade, topic_dist) in enumerate(zip(df['decade'], doc_topics)):
    for topic_id, prob in topic_dist:
        topic_strength[topic_id].append((decade, prob))

# 聚合每个时间段的主题强度
aggregated_strength = {i: {} for i in range(lda.num_topics)}

for topic_id, decade_probs in topic_strength.items():
    DF = pd.DataFrame(decade_probs, columns=['decade', 'prob'])
    df_grouped = DF.groupby('decade').mean()
    aggregated_strength[topic_id] = df_grouped

# 绘制每个主题的强度随时间段变化的折线图
plt.figure(figsize=(14, 8))
for topic_id, df_grouped in aggregated_strength.items():
    plt.plot(df_grouped.index, df_grouped['prob'], label=f'Topic {topic_id}')

plt.xlabel('Decade')
plt.ylabel('Topic Strength')
plt.title('Topic Strength Over Decades')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np

# 计算日期-主题强度
def get_document_topics(lda_model, corpus):
    doc_topics = lda_model.get_document_topics(corpus, minimum_probability=0)
    return doc_topics

doc_topics = get_document_topics(lda, corpus)

# 添加日期列并转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 创建时间段列，按五年分段
def get_decade(date):
    year = date.year
    period_start = (year // 5) * 5
    return f"{period_start}-{period_start + 4}"

df['decade'] = df['date'].apply(get_decade)

# 初始化主题强度数据结构
topic_strength = {i: [] for i in range(lda.num_topics)}

# 遍历文档，计算每篇文档的主题分布并按时间段聚合
for doc_id, (decade, topic_dist) in enumerate(zip(df['decade'], doc_topics)):
    for topic_id, prob in topic_dist:
        topic_strength[topic_id].append((decade, prob))

# 聚合每个时间段的主题强度
aggregated_strength = {i: {} for i in range(lda.num_topics)}

for topic_id, decade_probs in topic_strength.items():
    df_topic = pd.DataFrame(decade_probs, columns=['decade', 'prob'])
    
    # 转换时间段为数值类型
    df_topic['decade'] = df_topic['decade'].apply(lambda x: int(x.split('-')[0]))
    
    df_grouped = df_topic.groupby('decade').mean()
    aggregated_strength[topic_id] = df_grouped

# 创建3D图形
fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(111, projection='3d')

# 调整视角和背景
ax.view_init(elev=20, azim=45)  # 调整视角以更好展示三维图形
ax.set_facecolor('white')  # 设置背景为白色以提高可读性

# 设置颜色映射和线条透明度
colors = plt.cm.get_cmap('tab10', lda.num_topics)  # 使用colormap来分配不同的颜色
alpha_value = 0.8  # 设置线条透明度

# 绘制每个主题的三维折线图
for topic_id, df_grouped in aggregated_strength.items():
    x = df_grouped.index.values  # 时间段作为X轴
    y = np.array([topic_id] * len(x))  # 主题ID作为Y轴
    z = df_grouped['prob'].values  # 主题强度作为Z轴（高度）

    # 确保所有数据都是数值类型，并过滤无效数据
    valid_indices = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(z)
    x = x[valid_indices]
    y = y[valid_indices]
    z = z[valid_indices]

    # 使用不同的颜色和透明度绘制线条
    ax.plot(x, y, z, label=f'Topic {topic_id}', color=colors(topic_id), alpha=alpha_value)

# 添加网格和标签
ax.grid(True)
ax.set_xlabel('Decade')
ax.set_ylabel('Topic ID')
ax.set_zlabel('Topic Strength')
ax.set_title('Topic Strength Over Decades')

# 添加图例并优化显示
ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
plt.tight_layout()

# 显示图形
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np

# 计算日期-主题强度
def get_document_topics(lda_model, corpus):
    doc_topics = lda_model.get_document_topics(corpus, minimum_probability=0)
    return doc_topics

doc_topics = get_document_topics(lda, corpus)

# 添加日期列并转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 创建时间段列，按五年分段
def get_decade(date):
    year = date.year
    period_start = (year // 5) * 5
    return f"{period_start}-{period_start + 4}"

df['decade'] = df['date'].apply(get_decade)

# 初始化主题强度数据结构
topic_strength = {i: [] for i in range(lda.num_topics)}

# 遍历文档，计算每篇文档的主题分布并按时间段聚合
for doc_id, (decade, topic_dist) in enumerate(zip(df['decade'], doc_topics)):
    for topic_id, prob in topic_dist:
        topic_strength[topic_id].append((decade, prob))

# 聚合每个时间段的主题强度
aggregated_strength = {i: {} for i in range(lda.num_topics)}

for topic_id, decade_probs in topic_strength.items():
    df_topic = pd.DataFrame(decade_probs, columns=['decade', 'prob'])
    
    # 转换时间段为数值类型
    df_topic['decade'] = df_topic['decade'].apply(lambda x: int(x.split('-')[0]))
    
    df_grouped = df_topic.groupby('decade').mean()
    aggregated_strength[topic_id] = df_grouped

# 创建3D图形
fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(111, projection='3d')

# 准备数据用于表面填充
x_vals = []
y_vals = []
z_vals = []

for topic_id, df_grouped in aggregated_strength.items():
    x = df_grouped.index.values  # 时间段作为X轴
    y = np.array([topic_id] * len(x))  # 主题ID作为Y轴
    z = df_grouped['prob'].values  # 主题强度作为Z轴（高度）

    x_vals.append(x)
    y_vals.append(y)
    z_vals.append(z)

# 将数据转换为二维矩阵，适合使用 plot_surface
x_vals = np.array(x_vals)
y_vals = np.array(y_vals)
z_vals = np.array(z_vals)

# 使用 plot_surface 进行填充
surf = ax.plot_surface(x_vals, y_vals, z_vals, cmap='viridis', edgecolor='none', alpha=0.8)

# 添加颜色条
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# 设置轴标签和标题
ax.set_xlabel('Decade')
ax.set_ylabel('Topic ID')
ax.set_zlabel('Topic Strength')
ax.set_title('Topic Strength Over Decades (Surface Plot)')

# 调整视角
ax.view_init(elev=30, azim=120)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 计算日期-主题强度
def get_document_topics(lda_model, corpus):
    doc_topics = lda_model.get_document_topics(corpus, minimum_probability=0)
    return doc_topics

doc_topics = get_document_topics(lda, corpus)

# 添加日期列并转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 创建时间段列，按年份分段
def get_year(date):
    return date.year

df['year'] = df['date'].apply(get_year)

# 初始化主题强度数据结构
topic_strength = {i: [] for i in range(lda.num_topics)}

# 遍历文档，计算每篇文档的主题分布并按年份聚合
for doc_id, (year, topic_dist) in enumerate(zip(df['year'], doc_topics)):
    for topic_id, prob in topic_dist:
        topic_strength[topic_id].append((year, prob))

# 聚合每个年份的主题强度
aggregated_strength = {i: {} for i in range(lda.num_topics)}

for topic_id, year_probs in topic_strength.items():
    df_topic = pd.DataFrame(year_probs, columns=['year', 'prob'])
    df_grouped = df_topic.groupby('year').mean()
    aggregated_strength[topic_id] = df_grouped

# 创建图形
plt.figure(figsize=(14, 8))

# 设置颜色映射
colors = plt.cm.get_cmap('tab10', lda.num_topics)

# 绘制每个主题的折线图和填充区域
for topic_id, df_grouped in aggregated_strength.items():
    x = df_grouped.index.values  # 年份
    y = df_grouped['prob'].values  # 主题强度

    # 绘制折线图
    plt.plot(x, y, color=colors(topic_id), label=f'Topic {topic_id}')

    # 绘制填充区域
    plt.fill_between(x, y, color=colors(topic_id), alpha=0.3)

# 添加图例和标签
plt.xlabel('Year')
plt.ylabel('Topic Strength')
plt.title('Topic Strength Over Years (with Filled Areas)')
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)

# 显示图形
plt.show()
